In [1]:
import requests
from bs4 import BeautifulSoup

# 目標URL
url = 'https://sheethub.com/data.gov.tw/%E6%A8%B9%E6%9C%A8%E7%97%85%E8%9F%B2%E5%AE%B3%E8%B3%87%E6%96%99?page=23'

# 發送HTTP请求
response = requests.get(url)
response.raise_for_status()  # 如果请求失败，抛出异常

# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(response.content, 'html.parser')

# 提取表格数据
table = soup.find('table')
rows = table.find_all('tr')

# 解析表格头部以確定列的索引
headers = [th.get_text(strip=True) for th in rows[0].find_all('th')]
epidemic_name_index = headers.index('疫情名稱')
tree_species_index = headers.index('林木種類')
affected_city_index = headers.index('受害縣市')
planting_quantity_index = headers.index('種植數量')

# 存储提取的數據
data = []

# 遍历表格行，提取數據
for row in rows[1:]:  # 跳過標題
    cols = row.find_all('td')
    if len(cols) > max(epidemic_name_index, tree_species_index, affected_city_index, planting_quantity_index):
        epidemic_name = cols[epidemic_name_index].get_text(strip=True)
        tree_species = cols[tree_species_index].get_text(strip=True)
        affected_city = cols[affected_city_index].get_text(strip=True)
        planting_quantity = cols[planting_quantity_index].get_text(strip=True)
        
        data.append((epidemic_name, tree_species, affected_city, planting_quantity))

# 顯示數據
for entry in data:
    print(f'疫情名稱: {entry[0]}, 林木種類: {entry[1]}, 受害縣市: {entry[2]}, 種植數量  : {entry[3]}')

疫情名稱: 無褐根病, 林木種類: 樟樹, 受害縣市: 台北市, 種植數量  : 1株
疫情名稱: 無褐根病, 林木種類: 不知名樹殘根, 受害縣市: 台北市, 種植數量  : 1株
疫情名稱: 褐根病, 林木種類: 桃花心木, 受害縣市: 台南市, 種植數量  : 
疫情名稱: 地衣, 林木種類: 鳳凰木, 受害縣市: 南投縣, 種植數量  : 1株
疫情名稱: 生理性老化, 林木種類: 相思樹, 受害縣市: 新北市, 種植數量  : 1株
疫情名稱: 褐根病, 林木種類: 杜英, 受害縣市: 新北市, 種植數量  : 1株
疫情名稱: 褐根病, 林木種類: 台灣灤樹, 受害縣市: 新北市, 種植數量  : 
疫情名稱: 立地環境不佳, 林木種類: 樟樹, 受害縣市: 彰化縣, 種植數量  : 
疫情名稱: 樟白介殼蟲, 林木種類: 樟樹, 受害縣市: 台東縣, 種植數量  : 
疫情名稱: 烏心石幹綿蚜, 林木種類: 烏心石, 受害縣市: 台東縣, 種植數量  : 
疫情名稱: 生理性落葉, 林木種類: 茄冬樹, 受害縣市: 台東縣, 種植數量  : 
疫情名稱: 褐根病, 林木種類: 鳳凰木, 受害縣市: 新北市, 種植數量  : 1株
疫情名稱: 褐根病, 林木種類: 洋紫荊, 受害縣市: 新北市, 種植數量  : 3株
疫情名稱: 褐根病, 林木種類: 榕樹, 受害縣市: 台北市, 種植數量  : 
疫情名稱: 不當修剪, 林木種類: 五葉松, 受害縣市: 南投縣, 種植數量  : 
疫情名稱: 水份養份不足, 林木種類: 榕樹, 受害縣市: 屏東縣, 種植數量  : 1株
疫情名稱: 褐根病, 林木種類: 鳳凰木1, 受害縣市: 台南市, 種植數量  : 
疫情名稱: 過度修枝, 林木種類: 菩提樹, 受害縣市: 台南市, 種植數量  : 
疫情名稱: 紫膠介殼蟲, 林木種類: 小葉欖仁, 受害縣市: 台南市, 種植數量  : 
疫情名稱: 過度修枝, 林木種類: 赤松, 受害縣市: 彰化縣, 種植數量  : 
疫情名稱: 種植地點不當, 林木種類: 櫻花(八重櫻), 受害縣市: 彰化縣, 種植數量  : 
疫情名稱: 褐根病, 林木種類: 掌葉蘋婆, 受害縣市: 高雄市, 種植數量  : 
疫情名稱: 環境不適, 林木種類: 山櫻花, 受害縣市: 高雄

In [2]:
import pyodbc

# 連接到SQL Server
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};' #ODBC 驅動程式管理員
                          'Server=LAPTOP-M8ATTO8F\MSSQLSERVER_2019;'  #本機資料庫
                          'Database=TreePests;' 
                          'Trusted_Connection=yes;')  #連結資料庫

cursor = conn.cursor()
# 創建表格（如果表格不存在）
cursor.execute('''
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='tree' AND xtype='U')
    CREATE TABLE tree (
        id INT PRIMARY KEY IDENTITY(1,1),
        epidemic_name NVARCHAR(255),
        tree_species NVARCHAR(255),
        affected_city NVARCHAR(255),
        planting_quantity NVARCHAR(255)
    )
''')
data={}
# 插入數據
for entry in data:
    cursor.execute('''
                   INSERT INTO tree (epidemic_name, tree_species, affected_city, planting_quantity)
                   VALUES (?, ?, ?, ?)
                   ''', entry)


# 提交事務
conn.commit()

# 關閉連接
cursor.close()
conn.close()

<>:5: SyntaxWarning: invalid escape sequence '\M'
<>:5: SyntaxWarning: invalid escape sequence '\M'
C:\Users\vicky\AppData\Local\Temp\ipykernel_9276\1246327818.py:5: SyntaxWarning: invalid escape sequence '\M'
  'Server=LAPTOP-M8ATTO8F\MSSQLSERVER_2019;'  #本機資料庫


In [11]:
from flask import Flask, render_template
import pyodbc
# 連接到SQL Server
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};' #ODBC 驅動程式管理員
                          'Server=LAPTOP-M8ATTO8F\MSSQLSERVER_2019;'  #本機資料庫
                          'Database=TreePests;' 
                          'Trusted_Connection=yes;')  #連結資料庫

app = Flask(__name__)

@app.route('/')
def index():
    cursor = conn.execute('SELECT epidemic_name, tree_species, affected_city, planting_quantity FROM tree')
    trees = cursor.fetchall()
    conn.close()    
    return render_template('tree.html', trees=trees)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


<>:5: SyntaxWarning: invalid escape sequence '\M'
<>:5: SyntaxWarning: invalid escape sequence '\M'
C:\Users\vicky\AppData\Local\Temp\ipykernel_13664\1056773356.py:5: SyntaxWarning: invalid escape sequence '\M'
  'Server=LAPTOP-M8ATTO8F\MSSQLSERVER_2019;'  #本機資料庫
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Jul/2024 13:43:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2024 13:43:19] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from flask import Flask, render_template, url_for
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager
import pyodbc
import re
import os

app = Flask(__name__)

def get_data():
    # 添加中文字體
    fontManager.addfont('NotoSansTC-Regular.otf')
    plt.rc('font', family='Noto Sans TC')

    # 連接到SQL Server
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                          'Server=LAPTOP-M8ATTO8F\MSSQLSERVER_2019;'
                          'Database=TreePests;'
                          'Trusted_Connection=yes;')

    cursor = conn.cursor()

    # 從資料庫讀取數據
    cursor.execute('SELECT affected_city, planting_quantity FROM tree')
    rows = cursor.fetchall()

    # 關閉連接
    cursor.close()
    conn.close()

    # 準備數據
    cities = {}
    for row in rows:
        affected_city = row.affected_city
        planting_quantity_str = row.planting_quantity
        
        # 從字串中刪除非數字部分
        planting_quantity_digits = re.sub(r'\D', '', planting_quantity_str)
        
        # 檢查是否為空字串，如果是則賦值為0
        if planting_quantity_digits:
            planting_quantity = int(planting_quantity_digits)
        else:
            planting_quantity = 0
        
        if affected_city in cities:
            cities[affected_city] += planting_quantity
        else:
            cities[affected_city] = planting_quantity

    return cities

def save_plot():
    cities = get_data()

    # 準備繪圖數據
    city_names = list(cities.keys())
    quantities = list(cities.values())

    # 繪製長條圖
    plt.figure(figsize=(12, 8))
    bars = plt.barh(city_names, quantities, color='skyblue')
    plt.xlabel('種植數量', fontsize=14)
    plt.ylabel('受害縣市', fontsize=14)
    plt.title('受害縣市的種植數量分佈', fontsize=16)
    plt.tight_layout()

    # 添加數據標籤
    for bar in bars:
        plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{bar.get_width()}',
                 va='center', ha='left', fontsize=12)

    # 確保靜態目錄存在
    if not os.path.exists('static'):
        os.makedirs('static')

    # 保存圖像到文件
    plt.savefig(os.path.join('static', 'plot.png'))
    plt.close()

@app.route('/')
def index():
    save_plot()  # 确保在每次加载主页时生成和保存图表
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


<>:17: SyntaxWarning: invalid escape sequence '\M'
<>:17: SyntaxWarning: invalid escape sequence '\M'
C:\Users\vicky\AppData\Local\Temp\ipykernel_13664\2655356517.py:17: SyntaxWarning: invalid escape sequence '\M'
  'Server=LAPTOP-M8ATTO8F\MSSQLSERVER_2019;'
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Jul/2024 12:22:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2024 12:22:29] "GET /static/plot.png HTTP/1.1" 200 -
